In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import xgboost as xgb
import sklearn.metrics as metrics
import datetime

In [27]:
inputfolder = 'data'
datadf = pd.read_csv(os.path.join(inputfolder,"final_match_data.csv"))
datadf['datetime'] = pd.to_datetime(datadf['datetime'])
with open(os.path.join(inputfolder,'datacolumns.txt'),'r') as f:
    content = f.readlines()
datacols = [x.strip() for x in content]
print(datadf.dtypes)
display(datadf.head())

atpdata = pd.read_csv(os.path.join(inputfolder,"ATP_matches.csv"))
cols = atpdata.drop(['Winner','Loser','Tournament','Tournament_Date','Court_Surface','Round_Description'],axis=1).columns
for col in cols:
    atpdata.loc[:,col]=pd.to_numeric(atpdata[col],errors='coerce')
print(atpdata.dtypes)
atpdata['datetime'] = pd.to_datetime(atpdata['Tournament_Date'])
atpdata['year'] = pd.DatetimeIndex(atpdata['datetime']).year
def fillrank(row,col,ATPframe):
    if np.isnan(row[col]):
            year = row['year']
            tournament = row['Tournament']
            surface = row['Court_Surface']
            yeardf = ATPframe[ATPframe['year']==year] #i can do this slicing in one line, but im doing in two to allow for easier debugging
            tourndf = ATPframe[(ATPframe['Tournament']==tournament)&(ATPframe['Court_Surface']==surface)]
            maxrank = int(max([tourndf['Winner_Rank'].max(),tourndf['Loser_Rank'].max()]))
            return maxrank+1
    else:
        return row[col]
atpdata.loc[:,'Winner_Rank'] = atpdata.apply(fillrank,axis=1,args=('Winner_Rank',atpdata))
atpdata.loc[:,'Loser_Rank'] = atpdata.apply(fillrank,axis=1,args=('Loser_Rank',atpdata))

display(atpdata.head())

#only going to use matches from 2014 onwards, not ussing matches from 2012-13 so we have enough historic data for matches in 2014
atpHardOnly = atpdata.loc[(atpdata['Court_Surface']=='Hard')&(atpdata['year']>2013),['Winner','Loser','datetime']]
frame1,frame2 = train_test_split(atpHardOnly,test_size=.5)
frame1.columns = ['Player1','Player2','datetime']
frame1['Player1Win'] = 1

frame2.columns = ['Player2','Player1','datetime']
frame2['Player1Win'] = 0

traininput = pd.concat([frame1,frame2])
traininput.sort_index(inplace=True)
print(traininput['Player1Win'].mean())
display(traininput.head())

# splitting into X and Y
X = traininput.drop('Player1Win',axis=1)
Y = traininput['Player1Win']

Player                        object
Rank                         float64
Sets_Won                     float64
Games_Won                      int64
Aces                         float64
DoubleFaults                 float64
FirstServes_Won              float64
FirstServes_In               float64
SecondServes_Won             float64
SecondServes_In              float64
BreakPoints_Won              float64
BreakPoints                  float64
ReturnPoints_Won             float64
ReturnPoints_Faced           float64
TotalPoints_Won                int64
won_game?                      int64
FirstServes_ratio            float64
SecondServes_ratio           float64
BreakPoints_ratio            float64
ReturnPoints_ratio           float64
datetime              datetime64[ns]
year                           int64
Tournament                    object
Round_Description             object
Court_Surface                 object
Total_Serves                 float64
Aces%                        float64
S

,Player,Rank,Sets_Won,Games_Won,Aces,DoubleFaults,FirstServes_Won,FirstServes_In,SecondServes_Won,SecondServes_In,...,BreakPoints_ratio,ReturnPoints_ratio,datetime,year,Tournament,Round_Description,Court_Surface,Total_Serves,Aces%,ServesWon%
0,Edouard Roger-Vasselin,106.0,2.0,12,5.0,2.0,22.0,30.0,12.0,19.0,...,0.571429,0.423729,2012-01-02,2012,Chennai,First Round,Hard,49.0,0.102041,0.693878
1,Dudi Sela,83.0,2.0,12,2.0,0.0,14.0,17.0,11.0,16.0,...,0.428571,0.620690,2012-01-02,2012,Chennai,First Round,Hard,33.0,0.060606,0.757576
2,Go Soeda,120.0,2.0,19,6.0,1.0,48.0,64.0,19.0,39.0,...,0.454545,0.400000,2012-01-02,2012,Chennai,First Round,Hard,103.0,0.058252,0.650485
3,Yuki Bhambri,345.0,2.0,12,1.0,2.0,22.0,29.0,9.0,17.0,...,0.384615,0.548387,2012-01-02,2012,Chennai,First Round,Hard,46.0,0.021739,0.673913
4,Yuichi Sugita,235.0,2.0,12,3.0,1.0,37.0,51.0,11.0,27.0,...,0.428571,0.407407,2012-01-02,2012,Chennai,First Round,Hard,78.0,0.038462,0.615385


Winner                        object
Loser                         object
Tournament                    object
Tournament_Date               object
Court_Surface                 object
Round_Description             object
Winner_Rank                  float64
Loser_Rank                   float64
Retirement_Ind                 int64
Winner_Sets_Won              float64
Winner_Games_Won               int64
Winner_Aces                  float64
Winner_DoubleFaults          float64
Winner_FirstServes_Won       float64
Winner_FirstServes_In        float64
Winner_SecondServes_Won      float64
Winner_SecondServes_In       float64
Winner_BreakPoints_Won       float64
Winner_BreakPoints           float64
Winner_ReturnPoints_Won      float64
Winner_ReturnPoints_Faced    float64
Winner_TotalPoints_Won         int64
Loser_Sets_Won               float64
Loser_Games_Won                int64
Loser_Aces                   float64
Loser_DoubleFaults           float64
Loser_FirstServes_Won        float64
L

,Winner,Loser,Tournament,Tournament_Date,Court_Surface,Round_Description,Winner_Rank,Loser_Rank,Retirement_Ind,Winner_Sets_Won,...,Loser_FirstServes_In,Loser_SecondServes_Won,Loser_SecondServes_In,Loser_BreakPoints_Won,Loser_BreakPoints,Loser_ReturnPoints_Won,Loser_ReturnPoints_Faced,Loser_TotalPoints_Won,datetime,year
0,Edouard Roger-Vasselin,Eric Prodon,Chennai,2-Jan-12,Hard,First Round,106.0,97.0,0,2.0,...,33.0,13.0,26.0,1.0,3.0,15.0,49.0,49,2012-01-02,2012
1,Dudi Sela,Fabio Fognini,Chennai,2-Jan-12,Hard,First Round,83.0,48.0,0,2.0,...,32.0,5.0,26.0,0.0,1.0,8.0,33.0,30,2012-01-02,2012
2,Go Soeda,Frederico Gil,Chennai,2-Jan-12,Hard,First Round,120.0,102.0,0,2.0,...,70.0,18.0,35.0,2.0,4.0,36.0,103.0,99,2012-01-02,2012
3,Yuki Bhambri,Karol Beck,Chennai,2-Jan-12,Hard,First Round,345.0,101.0,0,2.0,...,33.0,13.0,29.0,2.0,3.0,15.0,46.0,43,2012-01-02,2012
4,Yuichi Sugita,Olivier Rochus,Chennai,2-Jan-12,Hard,First Round,235.0,67.0,0,2.0,...,32.0,13.0,22.0,1.0,7.0,30.0,78.0,62,2012-01-02,2012


0.499923629143119


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort

,Player1,Player1Win,Player2,datetime
5900,Albert Ramos-Vinolas,0,Edouard Roger-Vasselin,2014-01-01
5901,Guillermo Garcia-Lopez,1,Aleksandr Nedovyesov,2014-01-01
5902,Aljaz Bedene,1,Henri Laaksonen,2014-01-01
5903,Jiri Vesely,1,Jeevan Nedunchezhiyan,2014-01-01
5904,Benjamin Becker,1,Julian Reister,2014-01-01


In [31]:
#defining preprocessing functions (from wrangleDataAttempt2)
def days_difference(date1,date2):
            diff = date2-date1
            return diff.days

def get_player_stats(inputframe,harddf,datacols,playercol):
    #idea is to pass a panda dataframe with columns [playername, tournamentdate] and be able to return a dataframe with the stats
    length = inputframe.shape[0]
    historicnames = ['careeravg_'+x for x in datacols]
    ytdnames = ['ytd_'+x for x in datacols]
    colnames = [playercol,'datetime','Rank','ytd_Aces%_Stddev','ytd_ServesWon%_Stddev'] + historicnames+ytdnames
    outputframe = pd.DataFrame(index=range(0,length),columns=colnames)
    outputframe.loc[:,playercol] = inputframe[playercol]
    outputframe.loc[:,'datetime'] = inputframe['datetime']
    for index in range(0,length):
        #extract player name and date of game
        row = inputframe.iloc[index,:]
        playername = row[playercol]
        date = row['datetime']
        
        #grab only data for that player before that date
        tempdf = harddf[harddf['Player']==playername]
        tempdf.loc[:,'timedelta'] = tempdf['datetime'].apply(days_difference,args=(date,))
        tempdf = tempdf[tempdf['timedelta']>0]
        if tempdf.empty:
            continue
        ytddf = tempdf.loc[tempdf['timedelta']<=365]
#         display(ytddf.head())
        currank = tempdf.loc[tempdf['timedelta'].idxmin(),'Rank']
        historicframe = tempdf.loc[:,datacols].mean()
        historicframe.index = historicnames
        YTDframe = ytddf.loc[:,datacols].mean()
        YTDframe.index = ytdnames
        
        #code for debugging NaNs
#         if tempdf.isna().sum().sum()>0:
#             print(playername)
#             print("ytdframe shape: {}, missing values: {}".format(ytddf.shape,ytddf.isna().sum().sum()))
#             print(ytddf.isna().sum())
#             print("careerdf shape: {}, missing values: {}".format(tempdf.shape,tempdf.isna().sum().sum()))
#             print(tempdf.isna().sum())

        outputframe.loc[index,['Rank','ytd_Aces%_Stddev','ytd_ServesWon%_Stddev']] = [currank,ytddf['Aces%'].std(),ytddf['ServesWon%'].std()]
        outputframe.loc[index,historicnames]=historicframe
        outputframe.loc[index,ytdnames]=YTDframe
    return outputframe

def get_difference(frame1,frame2,colname=None, prefix1='',prefix2='',index1=None,index2=None):
    #check if index between frames are identical, if not, reset both
    idx_check = frame1.index.equals(frame2.index)
    if not(idx_check):
        frame1.reset_index(drop=True, inplace=True)
        frame2.reset_index(drop=True, inplace=True)
    if colname is None:
        colname = frame1.columns
    if frame1.shape[0] != frame2.shape[0]:
        raise ValueError('both frames must contain the same number of rows/n Frame1: %{}/mn Frame2: %{}'.format(frame1.shape[0],frame2.shape[0]))
    tempdiffcolnames = [x+"_diff" for x in colname]
    diffcolnames = list()
    if index1 is not None:
        diffcolnames.append(index1)
    if index2 is not None:
        diffcolnames.append(index2)
    diffcolnames = diffcolnames + tempdiffcolnames
    if idx_check:
        difframe = pd.DataFrame(index=frame1.index,columns=diffcolnames,data=0)
    else:
        difframe = pd.DataFrame(index=range(0,frame1.shape[0]),columns=diffcolnames,data=0)
    if index1 is not None:
        difframe[index1] = frame1[index1]
    if index2 is not None:
        difframe[index2] = frame2[index2]
    for col in colname:
        frame1name = prefix1+col
        frame2name = prefix2+col
        difframename = col+"_diff"
        difframe[difframename]=frame1[frame1name] - frame2[frame2name]
    return difframe

def convert_to_data(inputframe,matchdata,datacols,player1colname="Player1",player2colname="Player2"):
    player1frame = inputframe.loc[:,[player1colname,'datetime']]
    player2frame = inputframe.loc[:,[player2colname,'datetime']]
    
    player1df = get_player_stats(player1frame,matchdata,datacols,player1colname)
    player2df = get_player_stats(player2frame,matchdata,datacols,player2colname)
    
    #append columns indicating whether this is a players first recorded Hard surface ATPgame (if it's their first game, Rank will return NaNs)
    player1df['Player1fg'] = player1df['Rank'].isna()
    player2df['Player2fg'] = player2df['Rank'].isna()
    
    player1df = player1df[~player1df['Player1fg']].fillna(0)
    player2df = player2df[~player2df['Player2fg']].fillna(0)
#     if player1df.isna().sum().sum() >0:
#         display(player1df)
    
    #for debugging purposes
    if player1df.shape[0] != player2df.shape[0]:
        return (player1df,player2df)
    
    newdatacols = player1df.drop([player1colname,'datetime','Player1fg'],axis=1).columns
    outputdf = get_difference(player1df,player2df,newdatacols,index1=player1colname,index2=player2colname)
    outputdf.loc[:,'Player1fg'] = player1df['Player1fg']
    outputdf.loc[:,'Player2fg'] = player2df['Player2fg']
    outputdf.loc[:,'datetime'] = player1df['datetime']
    return outputdf

In [46]:
# X = convert_to_data(X,datadf,datacols)
frame1 = X[0]
frame2 = X[1]
print(frame1.shape)
print(frame2.shape)
inddif = frame1.index.difference(frame2.index)
display(frame1.head())

(6369, 48)
(6355, 48)


,Player1,datetime,Rank,ytd_Aces%_Stddev,ytd_ServesWon%_Stddev,careeravg_Sets_Won,careeravg_Games_Won,careeravg_Aces,careeravg_DoubleFaults,careeravg_FirstServes_Won,...,ytd_TotalPoints_Won,ytd_won_game?,ytd_FirstServes_ratio,ytd_SecondServes_ratio,ytd_BreakPoints_ratio,ytd_ReturnPoints_ratio,ytd_Total_Serves,ytd_Aces%,ytd_ServesWon%,Player1fg
0,0,0,75.0,0.018829,0.053715,1.153846,12.923077,3.884615,3.615385,35.423077,...,94.000000,0.400000,0.718995,0.523023,0.386053,0.365264,96.700000,0.049767,0.639944,False
1,0,0,74.0,0.031258,0.041567,1.000000,12.090909,3.909091,4.181818,32.909091,...,88.666667,0.166667,0.692580,0.470672,0.431878,0.342838,97.333333,0.074581,0.593801,False
2,0,0,106.0,0.041373,0.080112,1.200000,11.866667,7.066667,3.800000,30.866667,...,78.363636,0.363636,0.771801,0.483725,0.426512,0.373619,78.090909,0.118269,0.646099,False
3,0,0,82.0,0.000000,0.000000,0.500000,11.000000,5.500000,5.000000,39.000000,...,112.000000,0.000000,0.666667,0.483871,0.400000,0.278261,137.000000,0.072993,0.583942,False
4,0,0,73.0,0.033901,0.081979,0.916667,11.875000,7.208333,3.250000,31.541667,...,78.923077,0.230769,0.736017,0.465932,0.305003,0.309629,84.846154,0.082001,0.604934,False


In [ ]:
#training and then testing a basic model
Xtrain,Xtest,ytrain,ytest = train_test_split(X,Y,test_size=.3)
params ={'max_depth':6,
         'n_estimators':200,
         'silent':False,
         'njobs':0,
        }
model = xgb.XGBClassifier(**params)
model.fit(Xtrain,ytrain)

def get_scores(predy,ytest):
    import sklearn.metrics as metrics
    acc = metrics.accuracy_score(ytest,predy)
    prec = metrics.precision_score(ytest,predy)
    recall = metrics.recall_score(ytest,predy)
    auc = metrics.roc_auc_score(ytest,predy)
    return {'accuracy':acc,'precision':prec,'recall':recall,'auc':auc}
score = model.score(Xtest,ytest)
predy = model.predict(Xtest)
scoresdict = get_scores(predy,ytest)
print("="*200)
for key,value in scoresdict.items():
    print("{}: {}".format(key,value))

print("="*200)
for idx, col in enumerate(Xtrain.columns.tolist()):
    print("{} importance: {}".format(col,model.feature_importances_[idx]))
